In [1]:
import torch
import torch.nn as nn

“手撕” (Hand-coding) Batch Normalization (BN) 是深度学习面试中非常经典的高频题。

看似简单，但有两个核心坑（也是面试官最爱考察的点）：
**训练（Training）和推理（Inference）的逻辑完全不同。**
1. 训练时：Batch 的数据分布可能一直在变，为了让网络适应这种变化并防止梯度消失/爆炸，我们需要强制把当前 Batch 拉回到标准分布。
2. 推理时：输入可能只有一个样本（Batch Size = 1），计算不出方差（方差为0）；或者我们希望推理结果是确定性的（Deterministic），不应该受同一个 Batch 里其他样本的影响。所以必须使用全局统计量（Running Mean/Var）。


**Running Mean/Var 的动量更新。**
* 训练时 (Training):我们拥有一个 Batch 的数据（比如 32 张图片），我们可以直接算出这个 Batch 的均值 $\mu_{batch}$ 和方差 $\sigma^2_{batch}$ 来做归一化。
* 推理时 (Inference/Test): 可能会遇到单张图片输入 (Batch Size = 1)，此时无法计算方差（或者方差为0）。 即便 Batch Size > 1，我们也不希望推理结果受同一个 Batch 里其他图片的影响（我们需要确定性的结果）。
* 解决方案： 在训练过程中，我们利用每一个 Batch 的统计量，一点点地“估算”出整个训练数据集的全局均值和全局方差。这个估算的过程，就是动量更新

假设：$\hat{\mu}_{old}$ 是上一时刻保存的全局均值。$\mu_{batch}$ 是当前 Batch 算出来的均值。$m$ 是动量参数 (momentum)。公式如下：$$\hat{\mu}_{new} = (1 - m) \times \hat{\mu}_{old} + m \times \mu_{batch}$$同理，方差的更新也是一样：$$\hat{\sigma}^2_{new} = (1 - m) \times \hat{\sigma}^2_{old} + m \times \sigma^2_{batch}$$解读：新的全局统计量，是由“一大半的历史经验”加上“一小部分的当前新知”混合而成的。

### 这里实现一个针对全连接层（输入形状 [N, D]）的 BatchNorm1d。

In [3]:
class MyBatchNorm1d(nn.Module):
    def __init__(self, num_features,eps=1e-5, momentum=0.1,):
        super(MyBatchNorm1d, self).__init__()
        self.eps = eps
        self.momentum = momentum

        # 1. 可学习参数
        # gamma初始化为1；beta初始化为0
        self.gamma = nn.Parameter(torch.ones(num_features))
        self.beta = nn.Parameter(torch.zeros(num_features))

        # 2. 统计量(Running Statistics) - 不是可学习参数，不需要梯度注册为buffer,这样state_dict才可以保留下来
        # 我们默认数据是服从正态分布的
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_var', torch.ones(num_features))

    def forward(self, x):
        '''
        :param x: x shape: [Batch_Size, Num_Features]
        '''

        if self.training:
        # 训练模式
        #BatchNorm 的定义是：
        #对 每一个 feature（神经元 / 通道），
        #在 当前 batch 的所有样本上 计算均值和方差。
        # 所以dim=0,一定要弄清楚
            batch_mean = x.mean(dim=0)
            batch_var = x.var(dim=0,unbiased=False) # 深度学习中通常使用有偏估计

        # 关键点：更新全局的 running stats
        # 公式: running = (1 - momentum) * running + momentum * current
        # 注意：PyTorch 的 momentum 定义与常见的 SGD momentum 略有不同，
        # 这里 momentum=0.1 表示保留 90% 的历史，吸收 10% 的新值。
            with torch.no_grad():
                self.running_mean = (1-self.momentum) * self.running_mean + self.momentum * batch_mean
                self.running_var = (1-self.momentum) * self.running_var + self.momentum * batch_var

            #归一化用的统计量是当前batch的
            mean = batch_mean
            var = batch_var

        else:
            # === 推理模式 (Inference/Test) ===
            # 使用训练积累下来的全局统计量
            mean = self.running_mean
            var = self.running_var

        # 3. 执行归一化
        x_normalized = (x - mean) / torch.sqrt(var + self.eps)

        # 4 仿射变化
        out = self.gamma * x_normalized + self.beta
        return out

### 这里实现一个针对图片（输入形状 [N,C,H,W]）的 BatchNorm2d。
如果是卷积层 (N, C, H, W)，BN 是在 通道 (Channel) 维度上进行的。 这意味着同一个 Channel 的所有像素（包括不同 Batch 的，和同一张图不同位置的）共享同一个均值和方差。

In [5]:
'''
代码修改点
# 输入 x: [N, C, H, W]
# 计算均值时，要在 (N, H, W) 三个维度上求平均，只保留 C
batch_mean = x.mean(dim=(0, 2, 3), keepdim=True)
batch_var = x.var(dim=(0, 2, 3), keepdim=True, unbiased=False)
'''

class MyBatchNom2d(nn.Module):
    def __init__(self,num_features,eps=1e-5, momentum=0.1):
        super(MyBatchNom2d, self).__init__()
        self.eps = eps
        self.momentum = momentum

        #1.可学习参数 (c,)
        self.gamma = nn.Parameter(torch.ones(num_features))
        self.beta = nn.Parameter(torch.zeros(num_features))

        #2.统计量(Running Statistics) (c,)
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_var', torch.ones(num_features))

    def forward(self,x):
        '''
        :param x: x shape: [N, C, H, W]
        '''

        if self.training:
            # 关键点 1：计算均值和方差的维度
            # 我们希望对每个 Channel 单独归一化，所以要聚合 N, H, W (dim=0,2,3)
            # keepdim=True 让结果形状变为 [1, C, 1, 1]，方便后面直接广播计算
            batch_mean = x.mean(dim=(0, 2, 3), keepdim=True) # [1,C,1,1]
            batch_var = x.var(dim=(0, 2, 3), keepdim=True, unbiased=False)  # [1,C,1,1]

            # 关键点 2：更新全局 running stats
            with torch.no_grad():
                self.running_mean = (1-self.momentum) * self.running_mean + self.momentum * batch_mean.squeeze()  #(c,)
                self.running_var = (1-self.momentum) * self.running_var + self.momentum * batch_var.squeeze() #(c,)

            mean = batch_mean #[1,C,1,1]
            var = batch_var  # [1,C,1,1]

        else:
            # running_mean 形状是 [C]，需要 view 成 [1, C, 1, 1] 才能和 x 计算
            mean = self.running_mean.view(1, -1, 1, 1)
            var = self.running_var.view(1, -1, 1, 1)

        # 3. 执行归一化 [1, C, 1, 1]
        x_normalized = (x - mean) / torch.sqrt(var + self.eps)

        # 4.仿射变化
        out = x_normalized * self.gamma.view(1, -1, 1, 1) + self.beta.view(1, -1, 1, 1)
        return out




